# NLP Spam Detection Filter
__

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
# Initiate the SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spam").getOrCreate()

## Read the data

In [4]:
data = spark.read.csv("resources/smsspamcollection/SMSSpamCollection", inferSchema=True, sep='\t')

# Lets rename the default columns
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the data

In [7]:
# We will create a new feature called 'length'

from pyspark.sql.functions import length

data = data.withColumn('length', length(data['text']))

data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [8]:
# Lets compare the average length of ham and spam messages
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [9]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler

tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

## Create and Execute the NLP Pipeline

In [12]:
from pyspark.ml import Pipeline

data_prep_pipeline = Pipeline(stages=[ham_spam_to_num, tokenizer, stopremove, count_vec, idf, clean_up])

# Fit and transform the pipeline
clean_data = data_prep_pipeline.fit(data).transform(data)

# For model and training, we would only use the label and features
clean_data.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



## The Model

In [13]:
clean_data = clean_data.select(['label', 'features'])

# We will use NaiveBayes , but we can play around with others

from pyspark.ml.classification import NaiveBayes

# use default
nb = NaiveBayes()

# Do a train and test split

training, test = clean_data.randomSplit([0.7, 0.3])

# fit it to training
spam_predictor = nb.fit(training)

In [14]:
# Apply it to the test set

test_results = spam_predictor.transform(test)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1057.0050084033...|[1.0,1.3953084787...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-573.64026910332...|[1.0,1.9330794683...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1179.7459894265...|[1.0,1.4035380580...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-655.24749417486...|[1.0,9.2813370395...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-541.32387869249...|[1.0,1.0173854682...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1376.9596919056...|[1.0,1.1705056825...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-843.73382994358...|[1.0,1.1235350464...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1301.6288100521...|[1.0,2.8679556775...|       0.0|
|  0.0|(13424,[0,1,30,12...|[-598.89791998451...|[1.0,2.7028299749...|       0.0|
|  0.0|(13424,[0

In [15]:
# Evaluation of results

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()

print(f"Accuracy of the model : {acc_eval.evaluate(test_results)}")

Accuracy of the model : 0.9287541541345368
